In [5]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from datetime import datetime
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_metrics(stats_df):
    correct_predictions = stats_df[stats_df['ground_truth'] == stats_df['cnn_prediction']]
    accuracy = len(correct_predictions) / len(stats_df)
    precision = precision_score(stats_df['ground_truth'], stats_df['cnn_prediction'])
    recall = recall_score(stats_df['ground_truth'], stats_df['cnn_prediction'])
    f1 = f1_score(stats_df['ground_truth'], stats_df['cnn_prediction'])
    return accuracy, precision, recall, f1

def serialize_model(model, accuracy):
    if accuracy >= 0.80:
        current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        model_filename = f"trained_model_{current_datetime}.h5"
        model.save(model_filename)
        print("Model serialized successfully. Model saved as:", model_filename)
        return model_filename
    else:
        return None

def preprocess_png(image_path):
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (224, 224))
    preprocessed_image = np.expand_dims(resized_image, axis=0)
    return preprocessed_image

def detect_lines_cnn(image_path, model):
    preprocessed_image = preprocess_png(image_path)
    prediction = model.predict(preprocessed_image, verbose=0)[0][0]
    if prediction >= 0.5:
        return True
    else:
        return False

def find_latest_dictionary_csv(folder_path):
    csv_files = [f for f in os.listdir(folder_path) if f.startswith('dictionary') and f.endswith('.csv')]
    if not csv_files:
        print("No dictionary CSV file found.")
        return None
    latest_csv = max(csv_files)
    return latest_csv

def load_dictionary(folder_path):
    latest_dictionary_csv = find_latest_dictionary_csv(folder_path)
    if latest_dictionary_csv:
        dictionary_df = pd.read_csv(os.path.join(folder_path, latest_dictionary_csv))
        return dictionary_df
    else:
        print("Exiting program.")
        exit()

def read_png_data(folder_path, dictionary_df, model):
    filenames = []
    ground_truth = []
    cnn_predictions = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            matching_row = dictionary_df[dictionary_df['output'] == filename]
            if len(matching_row) == 0:
                print(f"No label found for filename: {filename}")
                continue
            label = matching_row.iloc[0]['label']
            ground_truth.append(label)
            cnn_prediction = detect_lines_cnn(image_path, model)
            cnn_predictions.append(cnn_prediction)
            filenames.append(filename)
    return pd.DataFrame({
        'filename': filenames,
        'ground_truth': ground_truth,
        'cnn_prediction': cnn_predictions
    })

def save_metrics_to_csv(metrics_list):
    current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    csv_filename = f"model_metrics_resnet_{current_datetime}.csv"
    metrics_df = pd.DataFrame(metrics_list, columns=['Accuracy', 'Precision', 'Recall', 'F1'])
    metrics_df.to_csv(csv_filename, index=False)
    print("Model metrics saved to:", csv_filename)

folder_path = './Data/fits_filtered8/'

metrics_list = []

for i in range(100):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

    dictionary_df = load_dictionary(folder_path)
    stats_df = read_png_data(folder_path, dictionary_df, model)

    accuracy, precision, recall, f1 = calculate_metrics(stats_df)

    print(i, ".: CNN Model Metrics:", accuracy, precision, recall, f1)
    model_filename = serialize_model(model, accuracy)
    metrics_list.append({'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1})

# Save the list of metrics to a CSV file
save_metrics_to_csv(metrics_list)


0 .: CNN Model Metrics: 0.47058823529411764 0.45454545454545453 0.625 0.5263157894736842


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1 .: CNN Model Metrics: 0.5294117647058824 0.0 0.0 0.0
2 .: CNN Model Metrics: 0.5588235294117647 1.0 0.0625 0.11764705882352941
3 .: CNN Model Metrics: 0.47058823529411764 0.47058823529411764 1.0 0.6399999999999999
4 .: CNN Model Metrics: 0.4411764705882353 0.45454545454545453 0.9375 0.6122448979591837
5 .: CNN Model Metrics: 0.5882352941176471 0.5555555555555556 0.625 0.5882352941176471
6 .: CNN Model Metrics: 0.47058823529411764 0.4666666666666667 0.875 0.608695652173913
7 .: CNN Model Metrics: 0.47058823529411764 0.47058823529411764 1.0 0.6399999999999999
8 .: CNN Model Metrics: 0.47058823529411764 0.4375 0.4375 0.4375
9 .: CNN Model Metrics: 0.47058823529411764 0.47058823529411764 1.0 0.6399999999999999
10 .: CNN Model Metrics: 0.47058823529411764 0.47058823529411764 1.0 0.6399999999999999


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


11 .: CNN Model Metrics: 0.5294117647058824 0.0 0.0 0.0


...

c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


93 .: CNN Model Metrics: 0.5294117647058824 0.0 0.0 0.0
94 .: CNN Model Metrics: 0.5 0.48484848484848486 1.0 0.653061224489796


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


95 .: CNN Model Metrics: 0.5294117647058824 0.0 0.0 0.0
96 .: CNN Model Metrics: 0.5 0.4827586206896552 0.875 0.6222222222222222
97 .: CNN Model Metrics: 0.5 0.48484848484848486 1.0 0.653061224489796
98 .: CNN Model Metrics: 0.5588235294117647 1.0 0.0625 0.11764705882352941
99 .: CNN Model Metrics: 0.5294117647058824 0.0 0.0 0.0
Model metrics saved to: model_metrics_resnet_2024-05-09_16-02-22.csv


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
